# Проект по SQL

**Цель проекта:**    
Компания решила быть на волне и купила крупный сервис для чтения книг по подписке, необходимо проанализировать базу данных книг для формирования ценностного предложение для нового продукта.

## Загрузка  и обзор данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
query = '''
SELECT * 
FROM books
LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''
SELECT * 
FROM publishers
LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = '''
SELECT * 
FROM reviews
LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [7]:
query = '''
SELECT * 
FROM ratings 
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query = '''
SELECT * 
FROM authors
LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица books**         
Содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.    

**Таблица authors**          
Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.  

**Таблица publishers**                  
Содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;    

**Таблица ratings**         
Содержит данные о пользовательских оценках книг:     

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.   

**Таблица reviews**           
Содержит данные о пользовательских обзорах на книги:    

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

## Посчитайте, сколько книг вышло после 1 января 2000 года

In [9]:
query = '''
SELECT count(*)
FROM books 
where publication_date>'2000-01-01'
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


- С 01.01.2020 вышло 819 книг

## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [10]:
query = '''
SELECT title, count(distinct rw.review_id), round(avg(rt.rating),2) as avg_r
FROM books as b
full join ratings as rt on b.book_id=rt.book_id
full join reviews as rw on b.book_id= rw.book_id
group by b.book_id
order by avg_r desc, count desc
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,count,avg_r
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


- Всего у нас 1000 книг. Самый высокий средний рейтинг и количество обзоров A Dirty Job (Grim Reaper #1)(рейтинг-5, 4 обзора)

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [11]:
query = '''
SELECT p.publisher, count(b.book_id) as cb
FROM books as b 
join publishers as p on p.publisher_id=b.publisher_id
where b.num_pages>50
group by p.publisher
order by cb desc
limit 1

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,cb
0,Penguin Books,42


- Больше всего книг выпустило издательство Penguin Books-42шт.

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [12]:
query = '''
SELECT a.author, round(avg(rating),3) as rating
from author as a
join books as b on b.author_id=a.author_id
join ratings as rt on rt.book_id=b.book_id
where b.book_id in (
select book_id
from ratings 
group by book_id
having count(rating_id) >50
order by count(rating_id) desc)
group by a.author
order by rating desc
limit 1

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.287


- Автор с самой выской средней оценкой книги - J.K. Rowling/Mary GrandPré(4.287)

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [13]:
query = '''
with rating as(
SELECT username, count(rating) as crt
from ratings
group by username
having count(rating)>48
order by crt desc),
obzors as (SELECT username, count(review_id) as crwi
from reviews
group by username
order by crwi desc)
select avg(o.crwi)
from rating
left join obzors as o on o.username=rating.username

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24

## Общий вывод

- С 01.01.2020 вышло 819 книг
- Всего у нас 1000 книг. Самый высокий средний рейтинг и количество обзоров A Dirty Job (Grim Reaper #1)(рейтинг-5, 4 обзора)
- Больше всего книг выпустило издательство Penguin Books-42шт.
- Автор с самой выской средней оценкой книги - J.K. Rowling/Mary GrandPré(4.287))
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24     

Рекомендации бизнесу:    
- повысить цену на книги с хорошим рейтингом
- запустить скидки на эти же книги(на первые месяцы работы)
- так же можно повысить цены на авторов с хорошим рейтингом
- возможно запросить скидку у издательств с которыми больше всего работаем
- так же возможно наладть связь с обзорщиками, чтобы они больше писали отзывов о книгах

In [14]:
query = '''
with aa as
(SELECT a.author, b.title,round(avg(rt.rating),2) as rating
FROM author as a
join books as b on b.author_id=a.author_id
join ratings as rt on rt.book_id=b.book_id
group by a.author, b.title),
bb as
(select b.title,b.book_id, count(rt.rating)
from books as b
join ratings as rt on rt.book_id=b.book_id
group by b.book_id,b.title
having count(rt.rating)>50)
select aa.author, aa.rating
from  bb
left join aa on aa.title=bb.title
order by aa.rating desc
limit 1

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.41
